# **Basic But Dumb Algorithm**
Why is it dumb? Uhm.. I'll answer it later

In [34]:
import random
import pandas as pd

In [35]:
content_categories = ["cat", "dog", "space"]

In [159]:
columns = ["video_id", "content_category", "like", "score" ]

In [37]:
content = []

#lets generate some content, in real world this is generated by the users
for i in range(100):
    video_id = f"video_{i}"
    content_category = content_categories[random.randint(0,2)] #for now we assume we already know "what" the content it about.
    like = 0 #0 indicates it is not liked by user
    score = like #if liked, then score is updated
    content.append([video_id, content_category, like, score ])


In [ ]:
df = pd.DataFrame(data=content, columns=columns)
df

,video_id,content_category,like,score
0,video_0,space,0,0
1,video_1,cat,0,0
2,video_2,space,0,0
3,video_3,dog,0,0
4,video_4,cat,0,0
...,...,...,...,...
95,video_95,cat,0,0
96,video_96,dog,0,0
97,video_97,space,0,0
98,video_98,space,0,0


## **Now lets try to make a very rudimentary system to mimic the interaction**
When you create a new account on any social media it will show the most popular content curated by mods or will throw random content at you and then it will learn based on how you interact. 

### **For now we are going to use random content strategy.**

In [ ]:

logs =[] #to keep a track of user's preferences

while True:
    len_content = len(df)
    random_int = random.randint(0, len_content-1)
    
    display_content_dict = df.iloc[random_int].to_dict()
    if display_content_dict["like"] == 0:
        response = input(f"Current Video: {display_content_dict['video_id']} is about {display_content_dict['content_category']}.\n Press y to like, or click enter to see new content")
        if response == "y":
            df.at[random_int, "like"] = 1
            df.at[random_int, "score"] += 1
            print(f"You liked {display_content_dict['video_id']} about {display_content_dict['content_category']}")
            print( df.iloc[random_int].to_dict())
            logs.append(df.iloc[random_int].to_dict())
        if response == "" or response == "n":
            df.at[random_int, "like"] = -1
            df.at[random_int, "score"] += -1
            print(f"You disliked {display_content_dict['video_id']} about {display_content_dict['content_category']}")
            print( df.iloc[random_int].to_dict())
            logs.append(df.iloc[random_int].to_dict())
        elif response == "q":
            break

    if display_content_dict["like"] == 1 or display_content_dict["like"] == -1:
        if input(f"Press q to quit") == "q":
            break


You liked video_59 about dog
You liked video_66 about dog
You disliked video_96 about dog
You liked video_94 about cat
You liked video_57 about dog
You disliked video_92 about cat
You disliked video_15 about space
You disliked video_76 about space
You liked video_83 about cat
You liked video_31 about dog


In [40]:
logs_df = pd.DataFrame(data=logs)
logs_df

,video_id,content_category,like,score
0,video_59,dog,1,1
1,video_66,dog,1,1
2,video_96,dog,-1,-1
3,video_94,cat,1,1
4,video_57,dog,1,1
5,video_92,cat,-1,-1
6,video_15,space,-1,-1
7,video_76,space,-1,-1
8,video_83,cat,1,1
9,video_31,dog,1,1


In [48]:
logs_describe_df = logs_df.groupby("content_category").describe()
logs_describe_df

like                                              score  \
                 count      mean       std  min  25%  50%  75%  max count   
content_category                                                            
cat                3.0  0.333333  1.154701 -1.0  0.0  1.0  1.0  1.0   3.0   
dog                5.0  0.600000  0.894427 -1.0  1.0  1.0  1.0  1.0   5.0   
space              2.0 -1.000000  0.000000 -1.0 -1.0 -1.0 -1.0 -1.0   2.0   

                                                               
                      mean       std  min  25%  50%  75%  max  
content_category                                               
cat               0.333333  1.154701 -1.0  0.0  1.0  1.0  1.0  
dog               0.600000  0.894427 -1.0  1.0  1.0  1.0  1.0  
space            -1.000000  0.000000 -1.0 -1.0 -1.0 -1.0 -1.0

### Here you can see, the user interacted with 10 videos out of which 3 were cat videos, 5 were dog videos, and 2 were space related videos. 

###  **Lets take a look at mean of all to get an idea of what user feels about the content categories.**

In [180]:
mean_dict = {}
#utlisingg our existing history
for category, data in logs_describe_df.iterrows():
    mean_dict[category] = float(data['score']['mean'])


mean_dict_for_df = {
    "category" : [],
    "mean": []
 }

for key, value in mean_dict.items():
    mean_dict_for_df["category"].append(key)
    mean_dict_for_df["mean"].append(value)

pd.DataFrame(mean_dict_for_df,).sort_values("mean", ascending= False)

,category,mean
1,dog,0.600000
0,cat,0.333333
2,space,-1.000000


### The user likes the dog content most, followed by about cat, and then at negative preference for space content.
### What can we infer from this? user is interested in dog content most, then cat and not in space. This means the algorithm should show more dog content, a bit of cat related content and avoid or show less of space.

### Currently we are not utilising this or making the algorithm personalised to user. Lets modify the algorithm a bit so that we will use the means of score per category

# **Making The Algorithm Personalised**


In [109]:
#lets create a user preference dictionary to keep track of categories and their preference levels.
user_preferences = {}

In [ ]:
#utlisingg our existing history
for category, data in logs_describe_df.iterrows():
    user_preferences[category] = float(data['score']['mean'])


preference_dict = {
    "category" : [],
    "preference_score": []
 }

for key, value in user_preferences.items():
    preference_dict["category"].append(key)
    preference_dict["preference_score"].append(value)

pd.DataFrame(preference_dict,).sort_values("preference_score", ascending= False)
#preference score is basically same as mean but we rebranded it as pereference_score

,category,preference_score
1,dog,0.600000
0,cat,0.333333
2,space,-1.000000


In [ ]:
#now lets create an algorithm which will recommend and update the preferences realtime

#step 1 is to rank content 
ranked_content = []

# we have about 100 videos in our content, and obviously we wont rank all 100 of them, i mean 1000s of videos and posts are
# uploaded on social medias, they wont rank each n everythink right? 
#so what we are gonna do is, we now know what user prefers, we are just gonna pick videos which ressemble to that topic
# so in our case user likes dogs and somewhat cats, and does not like space so there is no point of showing them space videos
# the goal is to maximise the time on app- cos thats how we will make money

In [ ]:
#lets see how many videos are of each topic
count = {} 
 
for index, data in df.iterrows():
    if data['content_category'] in count.keys():
        count[data['content_category']] += 1
    else :
         count[data['content_category']] = 1r

count_ls = {
    "category" : [],
    "count": []
 }

for key, value in count.items():
    count_ls["category"].append(key)
    count_ls["count"].append(value)

pd.DataFrame(count_ls,  )

,category,count
0,space,23
1,cat,42
2,dog,35


In [ ]:
# we have around 35 videos of dogs and 42 videos of cats. That already reduces our videos to work with from 100 to 77
# now based on user's preferences we will try to reduce it more, or rank it. 
# since user is more interested in dogs, we will show more number of dog videos compared to cat

# one more thing, we wont show all the 77 videos all at once.
# Just imagine in real life even after sorting and reducing, the number of dog videos will be still in hundreds of thousands 
# we will implement something called batch processing and lazy loading,
# basically take few, show them to user, load more in background when user is about to finish engaging with the
# content served before

# for this, we will take a batch size of 20, we will show 20 videos to user, and i will split the ratio as per preference
# and for now we wont load more, mostly because it is not strictly relevant to the topic of the video.

In [182]:
# lets find how many videos we need per category in a batch
batch_size = 20
count_video_per_category = {}
sum_of_preferences_all_categories = 0

for key, value in user_preferences.items():
    if value > 0 : 
        sum_of_preferences_all_categories += value

for key, value in user_preferences.items():
    if value > 0:
        count_video_per_category[key] = round(batch_size * user_preferences[key] / sum_of_preferences_all_categories)

count_video_per_category

count_video_per_category_dict = {
    "category" : [],
    "count": []
 }

for key, value in count_video_per_category.items():
    count_video_per_category_dict["category"].append(key)
    count_video_per_category_dict["count"].append(value)

pd.DataFrame(count_video_per_category_dict)


,category,count
0,cat,7
1,dog,13


In [ ]:
# so we need to find around 13 dog videos and 7 cat vidoes to serve to user
ranked_content = []
ranked_content_video_count_per_category = {}

#Step 1 : Get videos of only preferred category:
preferred_df = df.copy()
category_wise_videos_dict = {}

for index, data in preferred_df.iterrows():
    category = data['content_category']
    if category in user_preferences.keys():
        if user_preferences[category] < 0:
            preferred_df.drop(index, inplace= True)
        elif user_preferences[category] > 0:
            if category in category_wise_videos_dict.keys():
                category_wise_videos_dict[category].append(data)
            else:
                category_wise_videos_dict[category] = []
                category_wise_videos_dict[category].append(data)

preferred_df = preferred_df.reset_index(drop=True)

#Step 2 : Get the number of videos mentioned in count_video_per_category:
for key, value in count_video_per_category.items():
    i = 0
    
    for data in category_wise_videos_dict[key]:
        if i >= value: break
        ranked_content.append(data)
        i +=1

#Diversify the content
random.shuffle(ranked_content)
ranked_content_df = pd.DataFrame(ranked_content).reset_index()
ranked_content_df

,index,video_id,content_category,like,score
0,26,video_26,dog,0,0
1,4,video_4,cat,0,0
2,21,video_21,dog,0,0
3,1,video_1,cat,0,0
4,17,video_17,dog,0,0
5,11,video_11,cat,0,0
6,31,video_31,dog,0,0
7,24,video_24,dog,0,0
8,9,video_9,dog,0,0
9,14,video_14,cat,0,0


### Here you can see, we are getting now a more personalised content. But there is a big problem here and for that lets see the same thing at scale, with more topics/content_categories, and more interaction methods such as time spent, sharing to friends, commenting, etc.

